In [ ]:
#Создание консультанта

Имя: Вадим
Позиция: Специалист по концепциям маркетинга

Целевая группа: Малый и средний бизнесы, которые хотят улучшить свои продажи и повысить узнаваемость бренда.

Услуги/задачи: Консультация по концепциям маркетинга.



 База знаний для ответов на вопросы: https://docs.google.com/document/d/1OsG96Rjht9szmapyZrCdDIucDQIzDFpVr0xAmoKc38c/edit?usp=sharing



In [ ]:
# Установка необходимых библиотек
!pip install langchain
!pip install -U langchain-community
!pip install transformers
!pip install torch
!pip install gspread
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install requests


# Использование langchain для создания индекса и поиска ответов
import re
import requests
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

# Метод загрузки текстового документа в векторную базу знаний
def load_search_indexes(url):
    # Извлекаем document ID гугл документа из URL с помощью регулярных выражений
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)

    # Если ID не найден - генерируем исключение
    if match_ is None:
        raise ValueError('Неверный Google Docs URL')

    # Первый элемент в результате поиска
    doc_id = match_.group(1)

    # Скачиваем гугл документ по его ID в текстовом формате
    response = requests.get(f'https://docs.google.com/document/d//1OsG96Rjht9szmapyZrCdDIucDQIzDFpVr0xAmoKc38c/export?format=txt')

    # При неудачных статусах запроса будет вызвано исключение
    response.raise_for_status()

    # Извлекаем данные как текст
    text = response.text

    # Вызываем метод векторизации текста и сохранения в векторную базу данных
    return create_embedding(text)

# Метод для создания векторной базы данных
def create_embedding(text):
    # Создание документа из текста
    loader = TextLoader(text)
    documents = loader.load()

    # Разделение текста на фрагменты
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    # Создание эмбеддингов с использованием модели HuggingFace
    embeddings = HuggingFaceEmbeddings()

    # Создание векторной базы данных
    vectorstore = FAISS.from_documents(texts, embeddings)

    return vectorstore

# Загрузка индексов
url = 'https://docs.google.com/document/d/1OsG96Rjht9szmapyZrCdDIucDQIzDFpVr0xAmoKc38c/edit'
vectorstore = load_search_indexes(url)

# Загрузка модели и токенизатора
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "sberbank-ai/rugpt3small_based_on_gpt2"  # Используем модель на русском языке
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Создание функции для генерации ответа
def generate_response(prompt, vectorstore):
    # Поиск релевантного текста в векторной базе данных
    docs = vectorstore.similarity_search(prompt, k=1)

    # Если найден релевантный текст, используем его
    if docs:
        answer = docs[0].page_content
    else:
        # В противном случае генерируем ответ с помощью модели
        inputs = tokenizer.encode(prompt, return_tensors="pt")
        outputs = model.generate(inputs, max_length=150, num_return_sequences=1, attention_mask=inputs.ne(tokenizer.pad_token_id).float())
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

# Тестирование
prompt = "Привет, Вадим! Какая концепция лучше?"
response = generate_response(prompt, vectorstore)
print(response)

HTTPError: 404 Client Error: Not Found for url: https://docs.google.com/document/d//1OsG96Rjht9szmapyZrCdDIucDQIzDFpVr0xAmoKc38c/export?format=txt